# OTOKLAV DATA REVIEW

## Imports Libraries

In [ ]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
import cufflinks as cf
%matplotlib inline 

from scipy import stats


pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)
pd.set_option('display.width', 1000)

import warnings
warnings.filterwarnings("ignore")

## Import Local `utils1` Modul

In [ ]:
#  FONKSIYONLARIMIZI DERLEDIGIMIZ `UTILS` ADINDAKI MODULU BU NOTEBOOK ICINE IMPORT EDIYORUZ

import sys

# utils1.py dosyasının bulunduğu dizini ham dize olarak ekleyin
sys.path.append("C:/Users/duygu/Desktop/00-GitHub-Repo/00-1INTERNSHIP/INT1_PROJECT/UTILITIES/")

# 'utils1.py' dosyasında olusturulan Utils sınıfını import et
from utils1 import Utils, UtilsAnalysis

# utils1 icindeki tum class'lardan bir nesne oluştur
utils = Utils()
utils_analysis = UtilsAnalysis()

In [ ]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(utils)

In [ ]:
# utils nesnesindeki tüm fonksiyonları ve öznitelikleri listelemek için
dir(UtilsAnalysis)

## User Defined Funcs for Summary of Data

In [ ]:
# =============== User-Defined-Function ==========================

def rename_columns_by_position(df, new_column_names):
    """
    DataFrame'deki sütun adlarını pozisyona göre yeniden adlandırır.

    Parameters:
        df (pd.DataFrame): Sütun adlarını değiştirmek istediğiniz DataFrame.
        new_column_names (list): Yeni sütun adlarının listesi.

    Returns:
        pd.DataFrame: Yeni sütun adlarıyla güncellenmiş DataFrame.
    """
    # Sütun sayısının uyumlu olup olmadığını kontrol et
    if len(new_column_names) != len(df.columns):
        raise ValueError("Sütun sayısı ile yeni adların sayısı eşleşmiyor.")
    
    # Yeni sütun adlarını atama
    df.columns = new_column_names   
    return df


#================ Categorical Features Summary ===================
def object_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Kategorik sütunlar için özetleme
    object_df = df.select_dtypes(include='object')
    
    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=object_df.columns)

    summary_df['Dtype'] = object_df.dtypes
    summary_df['Counts'] = object_df.count()
    summary_df['Nulls'] = object_df.isnull().sum()
    summary_df['NullPercent'] = (object_df.isnull().sum() / obs) * 100
    summary_df['Top'] = object_df.apply(lambda x: x.mode().iloc[0] if not x.mode().empty else '-')
    summary_df['Frequency'] = object_df.apply(lambda x: x.value_counts().max() if not x.value_counts().empty else '-')
    summary_df['Uniques'] = object_df.nunique()

    # UniqueValues sütununu kontrol ederek ekleme (dize olarak)
    summary_df['UniqueValues'] = object_df.apply(
        lambda x: ', '.join(map(str, x.unique()[:10])) + '...' if x.nunique() > 10 else ', '.join(map(str, x.unique()))
    )

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df


#================ Numerical Features Summary ===================
def numeric_summary(df):
    obs = df.shape[0]
    duplicate_count = df.duplicated().sum()

    # Numerik sütunlar için özetleme
    numeric_df = df.select_dtypes(include=['float64', 'int64'])

    # Yeni bir boş DataFrame oluşturma
    summary_df = pd.DataFrame(index=numeric_df.columns)

    summary_df['Dtype'] = numeric_df.dtypes
    summary_df['Counts'] = numeric_df.count()
    summary_df['Nulls'] = numeric_df.isnull().sum()
    summary_df['NullPercent'] = (numeric_df.isnull().sum() / obs) * 100
    summary_df['Mean'] = numeric_df.mean()
    summary_df['Std'] = numeric_df.std()
    summary_df['Min'] = numeric_df.min()
    summary_df['25%'] = numeric_df.quantile(0.25)
    summary_df['50% (Median)'] = numeric_df.median()
    summary_df['75%'] = numeric_df.quantile(0.75)
    summary_df['Max'] = numeric_df.max()

    # DataFrame şekli ve tekrar eden satır sayısını ekrana yazdırma
    print(f'1. Data shape (rows, columns): {df.shape}')
    print(f'2. Number of duplicate rows: {duplicate_count}')
    return summary_df
    

#========== Get count and percentage of values for each column =================
def get_value_count(df, column_name):
    """
    This function calculates and returns a DataFrame with the value counts and 
    their corresponding percentages for a specified column in the DataFrame.
    """
    
    vc = df[column_name].value_counts()
    vc_norm = df[column_name].value_counts(normalize=True)
    
    vc = vc.rename_axis(column_name).reset_index(name='counts')
    vc_norm = vc_norm.rename_axis(column_name).reset_index(name='percent')
    vc_norm['percent'] = (vc_norm['percent'] * 100).map('{:.2f}%'.format)
    
    df_result = pd.concat([vc[column_name], vc['counts'], vc_norm['percent']], axis=1)
    return df_result


#============== Checks duplicates and drops them ==========================

def duplicate_values(df):
    print("Duplicate check...")
    num_duplicates = df.duplicated(subset=None, keep='first').sum()
    if num_duplicates > 0:
        print("There are", num_duplicates, "duplicated observations in the dataset.")
        df.drop_duplicates(keep='first', inplace=True)
        print(num_duplicates, "duplicates were dropped!")
        print("No more duplicate rows!")
    else:
        print("There are no duplicated observations in the dataset.")



# ========== User-Defined-Function for Missing Values ============
def missing_values(df):
    """This function calculates the missing values count and their percentage in a DataFrame."""

    missing_count = df.isnull().sum()
    value_count = df.isnull().count()
    missing_percentage = round(missing_count / value_count * 100, 2)
    
    # Format the percentage as '0.00%' with % symbol
    missing_percentage_formatted = missing_percentage.map("{:.2f}%".format)
    # Create a DataFrame to store the results
    missing_df = pd.DataFrame({"count": missing_count, "percentage": missing_percentage_formatted}) 
    return missing_df


# ========== Plotting Missing Values  ===========================
def na_ratio_plot(df):
    """Plots the ratio of missing values for each feature and prints the count of missing values."""
    
    sns.displot(df.isna().melt(value_name='Missing_data',var_name='Features')\
                ,y='Features',hue='Missing_data',multiple='fill',aspect=9/8)

    print(df.isna().sum()[df.isna().sum()>0])

    
    
#========== Detecting Anomalies ================================

def detect_anomalies(df, column_name):
    """
    Detects values with unusual (non-alphanumeric) characters in a column.
    Returns: list: Detected unusual character values.
    """
    # Get the unique values in the column
    unique_values = df[column_name].unique()    
    # Detect values with unusual characters (non-alphanumeric)
    unusual_characters = [val for val in unique_values if isinstance(val, str) and not val.isalnum()]
    
    # Return the list as a single string with values separated by commas
    return ', '.join(unusual_characters)


#========== Detecting Non-Numerical Characters ===========================

import re

def find_non_numeric_values(df, column_name):
    """
    Finds unique non-numeric values in a specified column of the DataFrame.
    """
    pattern = r'\D+'  # Pattern to match non-numeric characters
    # Find and flatten non-numeric values, then ensure uniqueness with set
    return set(re.findall(pattern, ' '.join(df[column_name].astype(str))))


#========================================================================
#======================================================================

# Exploratory Data Analysis (EDA)

## 2018

In [ ]:
df18 = pd.read_csv("df_otoklav_18.csv")
df_otoklav_18 = df18.copy()

df_otoklav_18.head()

In [ ]:
# 'TESLİM TARİHİ' sütununu DataFrame'den silme (ham verimizde yok)
#df_otoklav_18 = df_otoklav_18.drop('TESLİM TARİHİ', axis=1)

### Rename Columns

In [ ]:
# Yeni sütun adlarının listesi
new_column_names = [
    'satis_turu', 
    'siparis_no_is_emri_no_teklif_no', 
    'siparis_tarihi',
    'siparisi_veren',
    'kurulum_yeri',
    'il',
    'urun_modeli',
    'urun_adi',
    'litre',
    'seri_no',
    'teslim_suresi',
    'teslim_tarihi'
]

# DataFrame'in sütun adlarını yeniden adlandırma
df_otoklav_18 = rename_columns_by_position(df_otoklav_18,new_column_names)

df_otoklav_18.head()

In [ ]:
df_otoklav_18.info()

In [ ]:
# Numeric data yok
numeric_summary(df_otoklav_18)

In [ ]:
object_summary(df_otoklav_18)

- Satis_türü = 4 farklı tür var.
- Siparis_no_is_emri_no_teklif_no	hepsi unique
- Siparis_tarihi datetime çevirince 86 tanesini siliyor çözüm ?
- Kurulum_yeri hastane - üniversite(rektörlük) - tıp,göz... merkezi -ADSM bu veriler çıkarılabilir en çok talep hangi krumda
- il türkiye şehir ve ülkelerden oluşuyor buradan ülkeler feature çıkabilir.
- Litre sadece sayılar kalabilir (bu feature ne işe yarayacak)
- Seri_no unik değer
- Teslim_suresi gün bazında ve tarih bazında yazılan değerler var.Düzenlenmeli.



In [ ]:
get_value_count(df_otoklav_18, 'teslim_suresi')

In [ ]:
get_value_count(df_otoklav_18, 'litre')

In [ ]:
get_value_count(df_otoklav_18, 'il')

In [ ]:
get_value_count(df_otoklav_18, 'kurulum_yeri')

In [ ]:
get_value_count(df_otoklav_18, 'siparisi_veren')

In [ ]:
df_otoklav_18.info()

### Missings

In [ ]:
missing_values(df_otoklav_18)

### Duplicates

In [ ]:
duplicate_values(df_otoklav_18)

## 2019 

In [ ]:
df19 = pd.read_csv('df_otoklav_19.csv')
df_otoklav_19 = df19.copy()

df_otoklav_19.head()

In [ ]:
object_summary(df_otoklav_19)

## 2020

In [ ]:
df_otoklav_20 = pd.read_csv('df_otoklav_20.csv')
df_otoklav_20.head()

In [ ]:
object_summary(df_otoklav_19)

## 2021

In [ ]:
df_otoklav_21 = pd.read_csv('df_otoklav_21.csv')
df_otoklav_21.head()

In [ ]:
object_summary(df_otoklav_21)

## 2022 

In [ ]:
df_otoklav_22 = pd.read_csv('df_otoklav_22.csv')
df_otoklav_22.head()

In [ ]:
object_summary(df_otoklav_22)

## 2023 

In [ ]:
df_otoklav_23 = pd.read_csv('df_otoklav_23.csv')
df_otoklav_23.head()

In [ ]:
object_summary(df_otoklav_23)

## 2023_endüstriyel

In [ ]:
df_otoklav_23_Endust = pd.read_csv('df_otoklav_23_Endust.csv')
df_otoklav_23_Endust.head()

In [ ]:
object_summary(df_otoklav_23_Endust)

## 2024 

In [ ]:
df_otoklav_24 = pd.read_csv('df_otoklav_24.csv')
df_otoklav_24.head()

In [ ]:
object_summary(df_otoklav_24)